# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create data frame from csv file of weatherPy data
weather_path = "output_data\cities.csv"
weather_data = pd.read_csv(weather_path)
weather_data.head()

,Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,Airai,-8.9266,125.4092,73.89,83,100,1.25,TL,1610161726
1,2,Iraquara,-12.2486,-41.6194,69.46,68,0,3.13,BR,1610161731
2,3,Rikitea,-23.1203,-134.9692,78.80,75,70,11.03,PF,1610161735
3,4,Albany,42.6001,-73.9662,19.99,74,4,7.61,US,1610161739
4,5,Šalčininkai,54.3000,25.3833,27.00,93,90,3.44,LT,1610161743


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps, use lng and Lat for weight, add heatmap layer
gmaps.configure(api_key=g_key)
city_locations = weather_data[['Lat', 'Lng']]
humidity=weather_data["Humidity"].astype(float)

In [4]:
#create gmap figure, put center point to spain, add heat layer
fig=gmaps.figure(center=(40.21, -3.71), zoom_level=2.2)
#keep max heat to use for intensity of gmap heat layer
max_heat=weather_data["Humidity"].max()
heat_layer=gmaps.heatmap_layer(city_locations, weights=humidity, dissipating=False, max_intensity=max_heat, point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create new data frame with criteria on max temp and wind speed
criteria_weather=weather_data[(weather_data["Max Temp"]>70)&(weather_data["Max Temp"]<80)&(weather_data["Wind Speed"]<10)&(weather_data["Cloudiness"]==0)]
criteria_weather

,Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,24,Bajil,15.0584,43.2851,70.25,73,0,1.83,YE,1610161831
30,31,Say,13.1007,2.3689,77.00,18,0,6.91,NE,1610161865
100,101,Cooma,-36.2333,149.1333,75.20,35,0,3.44,AU,1610162172
134,135,Uruguaiana,-29.7547,-57.0883,71.60,68,0,4.61,BR,1610162347
140,141,Batemans Bay,-35.7167,150.1833,75.99,51,0,1.01,AU,1610162376
160,161,Caravelas,-17.7125,-39.2481,77.20,82,0,8.86,BR,1610162465
174,175,Porto Velho,-8.7619,-63.9039,75.20,94,0,1.14,BR,1610162523
213,214,Puri,19.8000,85.8500,71.60,64,0,4.61,IN,1610162699
237,238,Imbituba,-28.2400,-48.6703,77.00,88,0,3.44,BR,1610162800
250,251,Saint-Philippe,-21.3585,55.7679,78.80,57,0,3.44,RE,1610162863


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel_df data frame from some fields of weather data frame and add new coulmn to that
hotel_df=criteria_weather[["Id", "Lat", "Lng", "City", "Country"]].copy()
hotel_df["Hotel Name"]=np.nan
hotel_df

,Id,Lat,Lng,City,Country,Hotel Name
23,24,15.0584,43.2851,Bajil,YE,NaN
30,31,13.1007,2.3689,Say,NE,NaN
100,101,-36.2333,149.1333,Cooma,AU,NaN
134,135,-29.7547,-57.0883,Uruguaiana,BR,NaN
140,141,-35.7167,150.1833,Batemans Bay,AU,NaN
160,161,-17.7125,-39.2481,Caravelas,BR,NaN
174,175,-8.7619,-63.9039,Porto Velho,BR,NaN
213,214,19.8000,85.8500,Puri,IN,NaN
237,238,-28.2400,-48.6703,Imbituba,BR,NaN
250,251,-21.3585,55.7679,Saint-Philippe,RE,NaN


In [7]:
#for each row in hotel_df call google map api for nearest hotel in 5000 meter radius
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    ln=row["Lng"]
    search=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{ln}&radius=5000&keyword=hotel&key={g_key}"
    
    response_hotel=requests.get(search).json()
    if response_hotel["status"]=="ZERO_RESULTS":
        print(f"No hotel found in 5000 meter radius of {row['City']}....")
        print("--------------------------------------------------")
    elif response_hotel["status"]=="OK":
        hotel_name=response_hotel["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"]=hotel_name
        print(f"{hotel_name} found in {row['City']}")
        print("--------------------------------------------------")
hotel_df=hotel_df.dropna()
hotel_df

Yemen Star Tourist Hotel found in Bajil
--------------------------------------------------
No hotel found in 5000 meter radius of Say....
--------------------------------------------------
Alpine Hotel found in Cooma
--------------------------------------------------
Hotel Presidente found in Uruguaiana
--------------------------------------------------
Bay Breeze Motel found in Batemans Bay
--------------------------------------------------
Pousada dos Navegantes found in Caravelas
--------------------------------------------------
Oscar Hotel Executive found in Porto Velho
--------------------------------------------------
Hotel Puri Beach Resort found in Puri
--------------------------------------------------
Praia Hotel Imbituba found in Imbituba
--------------------------------------------------
Pinpin d'Amour found in Saint-Philippe
--------------------------------------------------
Port Lincoln Hotel found in Port Lincoln
--------------------------------------------------
No hot

,Id,Lat,Lng,City,Country,Hotel Name
23,24,15.0584,43.2851,Bajil,YE,Yemen Star Tourist Hotel
100,101,-36.2333,149.1333,Cooma,AU,Alpine Hotel
134,135,-29.7547,-57.0883,Uruguaiana,BR,Hotel Presidente
140,141,-35.7167,150.1833,Batemans Bay,AU,Bay Breeze Motel
160,161,-17.7125,-39.2481,Caravelas,BR,Pousada dos Navegantes
174,175,-8.7619,-63.9039,Porto Velho,BR,Oscar Hotel Executive
213,214,19.8000,85.8500,Puri,IN,Hotel Puri Beach Resort
237,238,-28.2400,-48.6703,Imbituba,BR,Praia Hotel Imbituba
250,251,-21.3585,55.7679,Saint-Philippe,RE,Pinpin d'Amour
321,322,-34.7333,135.8667,Port Lincoln,AU,Port Lincoln Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))